In [1]:
from pyspark.sql.functions import col, when, to_timestamp, unix_timestamp
from pyspark.sql.types import DoubleType
from  pyspark.sql.functions import abs, pow
import pyspark.sql.functions as F
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate()
# spark = SparkSession(sc)

In [2]:
from pyspark.sql.types import (StructField,StructType, BooleanType, DoubleType,LongType, IntegerType)
from pyspark.sql.types import *

In [3]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [4]:
# Définition du type : SPOINT

spoint_schema = [StructField('lat', FloatType(),True),
                 StructField('lon', FloatType(),True)]
spoint = StructType(fields=spoint_schema)

In [5]:
# Définition du type : SECTION

section_schema = [StructField('lat1', FloatType(),True),
                  StructField('lon1', FloatType(),True),
                  StructField('lat2', FloatType(),True),
                  StructField('lon2', FloatType(),True)]
section = StructType(fields=section_schema)

In [6]:
# Définition du type : SLINE

sline_schema = [StructField('rints', ArrayType(section),True)]
sline = StructType(fields=sline_schema)

In [7]:
# Définition du type : USPOINT

uspoint_schema = [StructField('t1', LongType(),True),
                  StructField('t2', LongType(),True),
                  StructField('lat1', FloatType(),True),
                  StructField('lon1', FloatType(),True),
                  StructField('lat2', FloatType(),True),
                  StructField('lon2', FloatType(),True)]
uspoint = StructType(fields=uspoint_schema)

In [8]:
# Définition du type : MSPOINT

mspoint_schema = [StructField('rints', ArrayType(uspoint),True)]
mspoint = StructType(fields=mspoint_schema)

In [9]:
# Définition du type : UINT

uint_schema = [StructField('val', IntegerType(),True),
               StructField('t1', LongType(),True),
               StructField('t2', LongType(),True)]
uint = StructType(fields=uint_schema)

In [10]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [11]:
# Définition du type : MINT

mint_schema = [StructField('units', ArrayType(uint),True)]
mint = StructType(fields=mint_schema)

In [12]:
# Définition du type : USTRING

ustring_schema = [StructField('val', StringType(),True),
               StructField('t1', LongType(),True),
               StructField('t2', LongType(),True)]
ustring = StructType(fields=ustring_schema)

In [13]:
# Définition du type : MSTRING

mstring_schema = [StructField('units', ArrayType(ustring),True)]
mstring = StructType(fields=mstring_schema)

In [14]:
# Définition du type : UREAL

ureal_schema = [StructField('a', FloatType(),True),
              StructField('b', FloatType(),True),
              StructField('c', FloatType(),True),
              StructField('r', BooleanType(),True),
              StructField('t1', LongType(),True),
              StructField('t2', LongType(),True)]
ureal = StructType(fields=ureal_schema)

In [15]:
# Définition du type : MREAL

mreal_schema = [StructField('units', ArrayType(ureal),True)]
mreal = StructType(fields=mreal_schema)

In [16]:
# Définition du type : USINT

usint_schema = [StructField('val', IntegerType(),True),
              StructField('interval', section,True)]
usint = StructType(fields=usint_schema)

In [17]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [18]:
# Définition du type : MSINT

msint_schema = [StructField('units', ArrayType(usint),False)]
msint = StructType(fields=msint_schema)

In [19]:
# Définition du type : USSTRING

usstring_schema = [StructField('val', StringType(),True),
              StructField('interval', section,True)]
usstring = StructType(fields=usstring_schema)

In [20]:
# Définition du type : MSSTRING

msstring_schema = [StructField('units', ArrayType(usstring),False)]
msstring = StructType(fields=msstring_schema)

In [21]:
# Définition du type : USREAL

usreal_schema = [StructField('a', FloatType(),True),
                 StructField('b', FloatType(),True),
                 StructField('c', FloatType(),True),
                 StructField('r', BooleanType(),True),
                 StructField('interval', section,True)]
usreal = StructType(fields=usreal_schema)

In [22]:
# Définition du type : MSREAL

msreal_schema = [StructField('units', ArrayType(usreal),False)]
msreal = StructType(fields=msreal_schema)

In [23]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [24]:
# Définition du type : INTIME

intime_schema = [StructField('val', FloatType(),True),
                 StructField('t1', IntegerType(),True)]
intime = StructType(fields=intime_schema)

In [25]:
# Définition du type : INSPOINT

inspoint_schema = [StructField('val', FloatType(),True),
                   StructField('sp', spoint,True)]
inspoint = StructType(fields=inspoint_schema)

In [26]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [27]:
# Lecture Data
df = spark.read.option("header",True).option("inferSchema",True).csv("VGP-week3-data.csv")
df.printSchema()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2-5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1-0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)



In [28]:
# Insert colonne Clé (profil spatial a notre DF)
df.printSchema()
df = df.rdd.map(lambda l :(l[0], l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11],(17+l[3])*31 + l[4])).toDF(StructType([
    StructField("kit_id",IntegerType()),
    StructField("participant_id",IntegerType()),
    StructField("time",StringType()),
    StructField("lat",DoubleType()),
    StructField("lon",DoubleType()),
    StructField("PM2-5",StringType()),
    StructField("PM10",StringType()),
    StructField("PM1-0",StringType()),
    StructField("NO2",StringType()),
    StructField("BC",StringType()),
    StructField("activity",StringType()),
    StructField("event",StringType()),
    StructField("cle",DoubleType()),
]))
df.show()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2-5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1-0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)

+------+--------------+-------------------+----------------+----------------+-----+----+-----+----+----+--------+-----+------------------+
|kit_id|participant_id|               time|             lat|             lon|PM2-5|PM10|PM1-0| NO2|  BC|activity|event|               cle|
+------+--------------+-------------------+----------------+----------------+-----+----+-----+----+----+--------+-----+------------------+
|    80|       9999964|2019-11-14 09:00:00|48.7717766666667|        2.006005| NULL|NULL| NULL|NULL|NULL|    NULL| NULL| 2040.9310

In [29]:
#On change les string "NULL" en null
df = df.withColumn(("BC"), when(col("BC") == "NULL", None).otherwise(col("BC")))
df = df.withColumn(("PM2-5"), when(col("PM2-5") == "NULL", None).otherwise(col("PM2-5")))
df = df.withColumn(("PM10"),  when(col("PM10") == "NULL",  None).otherwise(col("PM10")))
df = df.withColumn(("PM1-0"), when(col("PM1-0") == "NULL", None).otherwise(col("PM1-0")))
df = df.withColumn(("NO2"),   when(col("NO2") == "NULL",   None).otherwise(col("NO2")))
df = df.withColumn(("activity"), when(col("activity") == "NULL", None).otherwise(col("activity")))
df = df.withColumn(("event"), when(col("event") == "NULL", None).otherwise(col("event")))

In [30]:
df = df.withColumn('time',unix_timestamp('time', 'yyyy-MM-dd HH:mm:ss').alias('time'))
df.show()
df.printSchema()

+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+------------------+
|kit_id|participant_id|      time|             lat|             lon|PM2-5|PM10|PM1-0| NO2|  BC|activity|event|               cle|
+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+------------------+
|    80|       9999964|1573718400|48.7717766666667|        2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718410|48.7717766666667|        2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718420|48.7717766666667|        2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718430|48.7717766666667|        2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718440|48.7717766666667|        2.006005| null|null| null|null|

In [31]:
#Conversion des types PM2.5, BC ... en double
#Remarque dans notre fichier de donnée j'ai eliminer le "+00" a chaque fois
df = df.withColumn("PM2-5",df["PM2-5"].cast(FloatType()))
df = df.withColumn("PM10",df["PM10"].cast(FloatType()))
df = df.withColumn("PM1-0",df["PM1-0"].cast(FloatType()))
df = df.withColumn("NO2",df["NO2"].cast(FloatType()))
df = df.withColumn("BC",df["BC"].cast(FloatType()))
df = df.withColumn("lat",df["lat"].cast(FloatType()))
df = df.withColumn("lon",df["lon"].cast(FloatType()))
df = df.withColumn("time",df["time"].cast(LongType()))
df.printSchema()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: long (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- PM2-5: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- PM1-0: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- BC: float (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)
 |-- cle: double (nullable = true)



In [32]:
def change_column_names(columns):
    return [c.replace('-', '_') for c in columns]

df = df.toDF(*change_column_names(df.columns))
df.show()
df.count()

+------+--------------+----------+---------+---------+-----+----+-----+----+----+--------+-----+------------------+
|kit_id|participant_id|      time|      lat|      lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|               cle|
+------+--------------+----------+---------+---------+-----+----+-----+----+----+--------+-----+------------------+
|    80|       9999964|1573718400| 48.77178| 2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718410| 48.77178| 2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718420| 48.77178| 2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718430| 48.77178| 2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718440| 48.77178| 2.006005| null|null| null|null|null|    null| null| 2040.931081666668|
|    80|       9999964|1573718450| 48.77178| 2.006005| null|null| null|n

59972

In [33]:
df.write.csv('WriteCSV','overwrite')

In [34]:
liste_id = df.select('participant_id').distinct().rdd.map(lambda r: r[0]).collect()
colonnes_names = ['PM2_5','PM10','PM1_0','NO2','BC','activity','event']

In [35]:
# Affichage de tout les ID des participants 
print(liste_id)
print(colonnes_names)

[9999920, 9999955, 9999975, 9999936, 9999930, 9999960, 9999964, 9999962, 999992]
['PM2_5', 'PM10', 'PM1_0', 'NO2', 'BC', 'activity', 'event']


In [36]:
def get_MReal(id_particip, colonne):
    #On test si le participant et la colonne existent
    if (((id_particip in liste_id) or (id_particip=="*")) and (colonne in colonnes_names)):
        # On enleve les espace
        colonne = colonne.strip()
        # On recupere le bon participant (en prenant en compte si c'est * ou avec id), la colonne sans les null et on tri par rapport au timestamp 
        if id_particip == "*":
            df_one_participant = df.where(df[colonne].isNotNull()).sort("time")
        else : 
            df_one_participant = df.where((df[colonne].isNotNull()) & (df['participant_id'] == id_particip)).sort("time")
        
        # A partir de la je reprend ma methode
    
        #Creation d'une nouvelle DF temp pour faire la jointure et regrouper les tuples (1 avec 2 , 3 avec 4, ect...)
        temp = df_one_participant
        #On renome les colonnes de temp pour qu'il y'ait pas d'ambiguité
        temp = temp.withColumnRenamed('kit_id','kit_id_temp')
        temp = temp.withColumnRenamed('participant_id','participant_id_temp')
        temp = temp.withColumnRenamed('time','time_temp')
        temp = temp.withColumnRenamed('lat','lat_temp')
        temp = temp.withColumnRenamed('lon','lon_temp')
        temp = temp.withColumnRenamed('PM2_5','PM2_5_temp')
        temp = temp.withColumnRenamed('PM10','PM10_temp')
        temp = temp.withColumnRenamed('PM1_0','PM1_0_temp')
        temp = temp.withColumnRenamed('NO2','NO2_temp')
        temp = temp.withColumnRenamed('BC','BC_temp')
        temp = temp.withColumnRenamed('activity','activity_temp')
        temp = temp.withColumnRenamed('event','event_temp')
        
        #On decremente le time de temp pour pouvoir faire la jointure avec == (car j'ai pas pu le faire avec == time_temp + 10)
        temp = temp.withColumn('time_temp', temp.time_temp - 10)
        #On fait la jointure
        #On fais le rename ici car y'avais un bug de spark 
        df_one_participant = df_one_participant.withColumnRenamed('time','time')
        x = df_one_participant.join(temp, [temp['time_temp'] == df_one_participant['time'] ])
        # On decremente le time_temp pour revenir a un etat cohérent
        y = x.select(x.lat,x.lon,x['time'],x['time_temp'] + 10, x.lat_temp,x.lon_temp, x['PM2_5'], x['PM2_5_temp'], x['PM10'], x['PM10_temp'], x['PM1_0'], x['PM1_0_temp'], x['NO2'], x['NO2_temp'], x['BC'], x['BC_temp'], x['activity'], x['activity_temp'], x['event'], x['event_temp'])
        
        #Le map des differents colonnes  
        if (colonne == "PM2_5"): 
            rdd_pm25 = y.rdd.map(lambda r :(0.0, ((r[7]-r[6])/(r[3]-r[2])), (((r[6]*r[3])-(r[7]*r[2]))/((r[3]-r[2]))),False,r[2],r[3])).toDF()
            #return spark.createDataFrame(rdd_pm25,mreal)
            return rdd_pm25
        elif (colonne == "PM10"): 
            rdd_pm10 = y.select(y.columns).rdd.map(lambda r : (0.0, ((r[9]-r[8])/(r[3]-r[2])), (((r[8]*r[3])-(r[9]*r[2]))/((r[3]-r[2]))),False,r[2],r[3])).toDF()
            #return spark.createDataFrame(rdd_pm10,mreal)
            return rdd_pm10
        elif (colonne == "PM1_0"): 
            rdd_pm1_0 = y.select(y.columns).rdd.map(lambda r :(0.0, ((r[11]-r[10])/(r[3]-r[2])), (((r[10]*r[3])-(r[11]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])).toDF()
            #return spark.createDataFrame(rdd_pm1_0,mreal)
            return rdd_pm1_0
        elif (colonne == "NO2"): 
            rdd_no2 = y.select(y.columns).rdd.map(lambda r :  (0.0, ((r[13]-r[12])/(r[3]-r[2])), (((r[12]*r[3])-(r[13]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])).toDF()
            #return spark.createDataFrame(rdd_no2,mreal)
            return rdd_no2
        elif (colonne == "BC"): 
            rdd_bc = y.select(y.columns).rdd.map(lambda r :   (0.0, ((r[15]-r[14])/(r[3]-r[2])), (((r[14]*r[3])-(r[15]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])).toDF()
            #return spark.createDataFrame(rdd_bc,mreal)
            return rdd_bc
        elif (colonne == "activity"): 
            rdd_activity = y.select(y.columns).rdd.map(lambda r : (r[16], r[2], r[3])).toDF()
            #return spark.createDataFrame(rdd_activity, mstring)
            return rdd_activity
        elif (colonne == "event"): 
            rdd_event = y.select(y.columns).rdd.map(lambda r : (r[18], r[2], r[3])).toDF()
            #return spark.createDataFrame(rdd_event, mstring)
            return rdd_event
    else : 
        #print("Ce participant n'existe pas || ou la colonne n'existe pas")
        raise ValueError("Ce participant n'existe pas || ou la colonne n'existe pas")

    
a = get_MReal("*","PM2_5")
a.show()

+---+----+--------------+-----+----------+----------+
| _1|  _2|            _3|   _4|        _5|        _6|
+---+----+--------------+-----+----------+----------+
|0.0| 0.0|           2.0|false|1573805670|1573805680|
|0.0| 0.0|           1.0|false|1573805940|1573805950|
|0.0| 0.0|           1.0|false|1573807100|1573807110|
|0.0| 0.0|           2.0|false|1573810300|1573810310|
|0.0| 0.3| -4.72143088E8|false|1573810300|1573810310|
|0.0| 0.0|           2.0|false|1573811410|1573811420|
|0.0| 0.7|-1.101667985E9|false|1573811410|1573811420|
|0.0|-0.3|  4.72143428E8|false|1573811410|1573811420|
|0.0| 0.4| -6.29524559E8|false|1573811410|1573811420|
|0.0|-0.7| 1.101667996E9|false|1573811410|1573811420|
|0.0| 0.0|           9.0|false|1573811410|1573811420|
|0.0| 0.0|           3.0|false|1573816080|1573816090|
|0.0|-0.1|  1.57381611E8|false|1573816080|1573816090|
|0.0|-0.1|  1.57381611E8|false|1573816080|1573816090|
|0.0| 1.0|-1.573816077E9|false|1573816080|1573816090|
|0.0| 0.1| -1.57381606E8|fal

In [37]:
def get_USReal(id_particip, colonne):
    #On test si le participant et la colonne existent
    if (((id_particip in liste_id) or (id_particip=="*")) and (colonne in colonnes_names)):
        # On enleve les espace
        colonne = colonne.strip()
        # On recupere le bon participant (en prenant en compte si c'est * ou avec id), la colonne sans les null et on tri par rapport au timestamp 
        if id_particip == "*":
            df_one_participant = df.where(df[colonne].isNotNull()).sort("time")
        else : 
            df_one_participant = df.where((df[colonne].isNotNull()) & (df['participant_id'] == id_particip)).sort("time")
        
        # A partir de la je reprend ma methode
        #Creation d'une nouvelle DF temp pour faire la jointure et regrouper les tuples (1 avec 2 , 3 avec 4, ect...)
        temp = df_one_participant
        #On renome les colonnes de temp pour qu'il y'ait pas d'ambiguité
        temp = temp.withColumnRenamed('kit_id','kit_id_temp')
        temp = temp.withColumnRenamed('participant_id','participant_id_temp')
        temp = temp.withColumnRenamed('time','time_temp')
        temp = temp.withColumnRenamed('lat','lat_temp')
        temp = temp.withColumnRenamed('lon','lon_temp')
        temp = temp.withColumnRenamed('PM2_5','PM2_5_temp')
        temp = temp.withColumnRenamed('PM10','PM10_temp')
        temp = temp.withColumnRenamed('PM1_0','PM1_0_temp')
        temp = temp.withColumnRenamed('NO2','NO2_temp')
        temp = temp.withColumnRenamed('BC','BC_temp')
        temp = temp.withColumnRenamed('activity','activity_temp')
        temp = temp.withColumnRenamed('event','event_temp')
        temp = temp.withColumnRenamed('cle','cle_temp')
        #On decremente le time de temp pour pouvoir faire la jointure avec == (car j'ai pas pu le faire avec == time_temp + 10)
        temp = temp.withColumn('time_temp', temp.time_temp - 10)
        #On fait la jointure
        #On fais le rename ici car y'avais un bug de spark 
        df_one_participant = df_one_participant.withColumnRenamed('time','time')
        x = df_one_participant.join(temp, [temp['time_temp'] == df_one_participant['time'] ])
        # On decremente le time_temp pour revenir a un etat cohérent
        y = x.select(x.lat,x.lon,x['time'],x['time_temp'] + 10, x.lat_temp,x.lon_temp, x['PM2_5'], x['PM2_5_temp'], x['PM10'], x['PM10_temp'], x['PM1_0'], x['PM1_0_temp'], x['NO2'], x['NO2_temp'], x['BC'], x['BC_temp'], x['activity'], x['activity_temp'], x['event'], x['event_temp'], x['cle'], x['cle_temp'] - 0.000001) #On ajoute le 0.00001 pour ne pas avoir Division/0
        #Le map des differents colonnes  
        if (colonne == "PM2_5"): 
            rdd_pm25 = y.rdd.map(lambda r :(0.0, ((r[7]-r[6])/(r[21]-r[20])), (((r[6]*r[21])-(r[7]*r[20]))/((r[21]-r[20]))),False,r[20],r[21])).toDF()
            #return spark.createDataFrame(rdd_pm25,mreal)
            return rdd_pm25
        elif (colonne == "PM10"): 
            rdd_pm10 = y.select(y.columns).rdd.map(lambda r : (0.0, ((r[9]-r[8])/(r[21]-r[20])), (((r[8]*r[21])-(r[9]*r[20]))/((r[21]-r[20]))),False,r[20],r[21])).toDF()
            #return spark.createDataFrame(rdd_pm10,mreal)
            return rdd_pm10
        elif (colonne == "PM1_0"): 
            rdd_pm1_0 = y.select(y.columns).rdd.map(lambda r :(0.0, ((r[11]-r[10])/(r[21]-r[20])), (((r[10]*r[21])-(r[11]*r[20]))/((r[21]-r[20]))), False, r[20],r[21])).toDF()
            #return spark.createDataFrame(rdd_pm1_0,mreal)
            return rdd_pm1_0
        elif (colonne == "NO2"): 
            rdd_no2 = y.select(y.columns).rdd.map(lambda r :  (0.0, ((r[13]-r[12])/(r[21]-r[20])), (((r[12]*r[21])-(r[13]*r[20]))/((r[21]-r[20]))), False, r[20],r[21])).toDF()
            #return spark.createDataFrame(rdd_no2,mreal)
            return rdd_no2
        elif (colonne == "BC"): 
            rdd_bc = y.select(y.columns).rdd.map(lambda r :   (0.0, ((r[15]-r[14])/(r[21]-r[20])), (((r[14]*r[21])-(r[15]*r[20]))/((r[21]-r[20]))), False, r[20],r[21])).toDF()
            #return spark.createDataFrame(rdd_bc,mreal)
            return rdd_bc
        elif (colonne == "activity"): 
            rdd_activity = y.select(y.columns).rdd.map(lambda r : (r[16], r[21], r[20])).toDF()
            #return spark.createDataFrame(rdd_activity, mstring)
            return rdd_activity
        elif (colonne == "event"): 
            rdd_event = y.select(y.columns).rdd.map(lambda r : (r[18], r[21], r[20])).toDF()
            #return spark.createDataFrame(rdd_event, mstring)
            return rdd_event
    else : 
        #print("Ce participant n'existe pas || ou la colonne n'existe pas")
        raise ValueError("Ce participant n'existe pas || ou la colonne n'existe pas")

    
a = get_USReal("*","PM2_5")
a.show()

+---+-------------------+-------------------+-----+------------------+------------------+
| _1|                 _2|                 _3|   _4|                _5|                _6|
+---+-------------------+-------------------+-----+------------------+------------------+
|0.0|               -0.0|                2.0|false| 2041.969001666668|2041.9690006666679|
|0.0|               -0.0|                1.0|false|2041.9672149999992|2041.9671606666677|
|0.0|                0.0|                1.0|false|        2041.96987|2041.9702190000007|
|0.0|               -0.0|                2.0|false|2041.9705983333324|2041.9705973333323|
|0.0| -92.48982612442346|  188863.5055910349|false|2041.9705983333324|2041.9381623333343|
|0.0|               -0.0|                2.0|false|2041.9696883333322|2041.9622856666656|
|0.0|  757.3844987731238|-1546554.1889082526|false|2041.9696883333322|2041.9789306666667|
|0.0| -177.5252973528344| 362501.96194624814|false|2041.9453866666654|2041.9622856666656|
|0.0| 119.

In [38]:
def get_USPoint(id_particip, colonne):
    #On test si le participant et la colonne existent
    if (((id_particip in liste_id) or (id_particip=="*")) and (colonne in colonnes_names)):
        # On enleve les espace
        colonne = colonne.strip()
        # On recupere le bon participant (en prenant en compte si c'est * ou avec id), la colonne sans les null et on tri par rapport au timestamp 
        if id_particip == "*":
            df_one_participant = df.where(df[colonne].isNotNull()).sort("time")
        else : 
            df_one_participant = df.where((df[colonne].isNotNull()) & (df['participant_id'] == id_particip)).sort("time")
        
        # A partir de la je reprend ma methode
        #Creation d'une nouvelle DF temp pour faire la jointure et regrouper les tuples (1 avec 2 , 3 avec 4, ect...)
        temp = df_one_participant
        #On renome les colonnes de temp pour qu'il y'ait pas d'ambiguité
        temp = temp.withColumnRenamed('kit_id','kit_id_temp')
        temp = temp.withColumnRenamed('participant_id','participant_id_temp')
        temp = temp.withColumnRenamed('time','time_temp')
        temp = temp.withColumnRenamed('lat','lat_temp')
        temp = temp.withColumnRenamed('lon','lon_temp')
        temp = temp.withColumnRenamed('PM2_5','PM2_5_temp')
        temp = temp.withColumnRenamed('PM10','PM10_temp')
        temp = temp.withColumnRenamed('PM1_0','PM1_0_temp')
        temp = temp.withColumnRenamed('NO2','NO2_temp')
        temp = temp.withColumnRenamed('BC','BC_temp')
        temp = temp.withColumnRenamed('activity','activity_temp')
        temp = temp.withColumnRenamed('event','event_temp')
        temp = temp.withColumnRenamed('cle','cle_temp')
        #On decremente le time de temp pour pouvoir faire la jointure avec == (car j'ai pas pu le faire avec == time_temp + 10)
        temp = temp.withColumn('time_temp', temp.time_temp - 10)
        #On fait la jointure
        #On fais le rename ici car y'avais un bug de spark 
        df_one_participant = df_one_participant.withColumnRenamed('time','time')
        x = df_one_participant.join(temp, [temp['time_temp'] == df_one_participant['time'] ])
        # On decremente le time_temp pour revenir a un etat cohérent
        y = x.select(x.lat,x.lon,x['time'],x['time_temp'] + 10, x.lat_temp,x.lon_temp, x['PM2_5'], x['PM2_5_temp'], x['PM10'], x['PM10_temp'], x['PM1_0'], x['PM1_0_temp'], x['NO2'], x['NO2_temp'], x['BC'], x['BC_temp'], x['activity'], x['activity_temp'], x['event'], x['event_temp'], x['cle'], x['cle_temp'] - 0.000001) #On ajoute le 0.00001 pour ne pas avoir Division/0
        #Le map des differents colonnes  
        rdd_all = y.rdd.map(lambda r :(r[2],r[3], r[20], r[21])).toDF()
        return rdd_all
    else : 
        #print("Ce participant n'existe pas || ou la colonne n'existe pas")
        raise ValueError("Ce participant n'existe pas || ou la colonne n'existe pas")

    
a = get_USPoint("*","PM1_0")
a.show()

+----------+----------+------------------+------------------+
|        _1|        _2|                _3|                _4|
+----------+----------+------------------+------------------+
|1573805670|1573805680| 2041.969001666668|2041.9690006666679|
|1573805940|1573805950|2041.9672149999992|2041.9671606666677|
|1573807100|1573807110|        2041.96987|2041.9702190000007|
|1573810300|1573810310|2041.9705983333324|2041.9705973333323|
|1573810300|1573810310|2041.9705983333324|2041.9381623333343|
|1573811410|1573811420|2041.9696883333322|2041.9622856666656|
|1573811410|1573811420|2041.9696883333322|2041.9789306666667|
|1573811410|1573811420|2041.9453866666654|2041.9622856666656|
|1573811410|1573811420|2041.9453866666654|2041.9789306666667|
|1573811410|1573811420|2041.9787083333322|2041.9622856666656|
|1573811410|1573811420|2041.9787083333322|2041.9789306666667|
|1573816080|1573816090|2040.9309416666656|2040.9309406666655|
|1573816080|1573816090|2040.9309416666656|2041.9706173333334|
|1573816

In [40]:
def MIN_Mreal(Mreal):
    if Mreal in colonnes_names : 
        df_operation = get_MReal("*",Mreal)
        df_operation = df_operation.withColumn(
            ("min"),
            when(
                col('_2') > 0,
                df_operation[1] * df_operation[5] + df_operation[2]
            ).otherwise((df_operation[1] * df_operation[4]) + df_operation[2])
        )
        return df_operation.describe('min').show()
t11 = time.time()
MIN_Mreal("PM2_5")
print ("Seconde : ", time.time() - t11)

+-------+-----------------+
|summary|              min|
+-------+-----------------+
|  count|            18404|
|   mean|8.201912627650769|
| stddev|8.796360304411078|
|    min|              0.0|
|    max|94.00000190734863|
+-------+-----------------+

Seconde :  194.72412967681885


In [41]:
def MAX_Mreal(Mreal):
    if Mreal in colonnes_names : 
        df_operation = get_MReal("*",Mreal)
        df_operation = df_operation.withColumn(
            ("max"),
            when(
                col('_2') > 0,
                df_operation[1] * df_operation[5] + df_operation[2]
            ).otherwise((df_operation[1] * df_operation[4]) + df_operation[2])
        )
        return df_operation.describe('max').show()
    
t11 = time.time()
MAX_Mreal("PM2_5")
print ("Seconde : ", time.time() - t11)

+-------+-----------------+
|summary|              max|
+-------+-----------------+
|  count|            18404|
|   mean|8.201912627650769|
| stddev|8.796360304411078|
|    min|              0.0|
|    max|94.00000190734863|
+-------+-----------------+

Seconde :  176.3342161178589


In [42]:
def MEAN_Mreal(Mreal):
    if Mreal in colonnes_names : 
        df_operation = get_MReal("*",Mreal)
        df_operation_1 = df_operation.withColumn('mean',(df_operation[0]/3)*abs(pow(df_operation[5],3) - pow(df_operation[4],3)) + (df_operation[1]/3)*abs(pow(df_operation[5],2) - pow(df_operation[4],2)) + df_operation[2]*abs(df_operation[5]-df_operation[4]))
        df_operation_2 = df_operation_1.withColumn('interval', abs(df_operation[5] - df_operation[4]))
        interval_sum = df_operation_2.select(F.sum('interval')).collect()[0][0]
        mean_sum = df_operation_2.select(F.sum('mean')).collect()[0][0]
        r = mean_sum/interval_sum
        #print(interval_sum.collect()[0][0], mean_sum.collect()[0][0])
        #df_operation.printSchema()
        return r
t11 = time.time()
print(MEAN_Mreal("PM2_5"))
print ("Seconde : ", time.time() - t11)

983716.7964051273
Seconde :  313.87360167503357


In [43]:
#Test Cell
x='buffalo'    
exec("%s = %s" % (x,'5'))
buffalo = a
buffalo.show()

+----------+----------+------------------+------------------+
|        _1|        _2|                _3|                _4|
+----------+----------+------------------+------------------+
|1573805670|1573805680| 2041.969001666668|2041.9690006666679|
|1573805940|1573805950|2041.9672149999992|2041.9671606666677|
|1573807100|1573807110|        2041.96987|2041.9702190000007|
|1573810300|1573810310|2041.9705983333324|2041.9705973333323|
|1573810300|1573810310|2041.9705983333324|2041.9381623333343|
|1573811410|1573811420|2041.9696883333322|2041.9622856666656|
|1573811410|1573811420|2041.9696883333322|2041.9789306666667|
|1573811410|1573811420|2041.9453866666654|2041.9622856666656|
|1573811410|1573811420|2041.9453866666654|2041.9789306666667|
|1573811410|1573811420|2041.9787083333322|2041.9622856666656|
|1573811410|1573811420|2041.9787083333322|2041.9789306666667|
|1573816080|1573816090|2040.9309416666656|2040.9309406666655|
|1573816080|1573816090|2040.9309416666656|2041.9706173333334|
|1573816

In [44]:
def atInstant_Mreal(Mreal, t):
    print("t = ", t)
    if Mreal in colonnes_names : 
        df_operation = get_MReal("*",Mreal)
        
        df_operation = df_operation.filter((col('_5')<= t) & (col('_6')>= t))
        #df_operation.show()
        #df_operation = df_operation.filter(col('_6')>= t)
        #df_operation.show()
        #df_operation.show()
        #df_operation.show()
        df_operation = df_operation.withColumn('valeur', (df_operation[1]*t) + df_operation[2])
        valuee = df_operation.select(df_operation['valeur']).collect()
        print(valuee)
        value = df_operation.select(df_operation['valeur']).collect()[0][0]
        print(value)
        d = [(value,t)]
        #print(d)
        result = spark.createDataFrame(d, intime)
        return result.show()
atInstant_Mreal("PM2_5", 1573811412)
#a.show()

t =  1573811412
[Row(valeur=2.0), Row(valeur=3.3999998569488525), Row(valeur=4.400000035762787), Row(valeur=5.800000071525574), Row(valeur=7.6000001430511475), Row(valeur=9.0)]
2.0
+---+----------+
|val|        t1|
+---+----------+
|2.0|1573811412|
+---+----------+



In [45]:
get_USReal("*","PM2_5").show()

+---+-------------------+-------------------+-----+------------------+------------------+
| _1|                 _2|                 _3|   _4|                _5|                _6|
+---+-------------------+-------------------+-----+------------------+------------------+
|0.0|               -0.0|                2.0|false| 2041.969001666668|2041.9690006666679|
|0.0|               -0.0|                1.0|false|2041.9672149999992|2041.9671606666677|
|0.0|                0.0|                1.0|false|        2041.96987|2041.9702190000007|
|0.0|               -0.0|                2.0|false|2041.9705983333324|2041.9705973333323|
|0.0| -92.48982612442346|  188863.5055910349|false|2041.9705983333324|2041.9381623333343|
|0.0|               -0.0|                2.0|false|2041.9696883333322|2041.9622856666656|
|0.0|  757.3844987731238|-1546554.1889082526|false|2041.9696883333322|2041.9789306666667|
|0.0| -177.5252973528344| 362501.96194624814|false|2041.9453866666654|2041.9622856666656|
|0.0| 119.

In [46]:
def MIN_USReal(USReal):
    if USReal in colonnes_names : 
        df_operation = get_USReal("*",USReal)
        df_operation = df_operation.withColumn(
            ("min"),
            when(
                col('_2') < 0,
                df_operation[1] * df_operation[5] + df_operation[2]
            ).otherwise((df_operation[1] * df_operation[4]) + df_operation[2])
        )
        return df_operation.describe('min').show()
t11 = time.time()
MIN_USReal("PM2_5")
print ("Seconde : ", time.time() - t11)

+-------+--------------------+
|summary|                 min|
+-------+--------------------+
|  count|               18404|
|   mean|   6.359704410958423|
| stddev|   7.885299417430125|
|    min|-1.16415321826934...|
|    max|   94.00000000092147|
+-------+--------------------+

Seconde :  182.1164071559906


In [47]:
def MAX_USReal(USReal):
    if USReal in colonnes_names : 
        df_operation = get_USReal("*",USReal)
        df_operation = df_operation.withColumn(
            ("max"),
            when(
                col('_2') > 0,
                df_operation[1] * df_operation[5] + df_operation[2]
            ).otherwise((df_operation[1] * df_operation[4]) + df_operation[2])
        )
        return df_operation.describe('max').show()
t11 = time.time()
MAX_USReal("PM2_5")
print ("Seconde : ", time.time() - t11)

+-------+--------------------+
|summary|                 max|
+-------+--------------------+
|  count|               18404|
|   mean|   6.344327319021883|
| stddev|   7.857120126104022|
|    min|-4.54747350886464...|
|    max|   94.00000000092147|
+-------+--------------------+

Seconde :  186.97272062301636


In [66]:
#Test Cell
from pyspark.sql import Row
titi=sc.parallelize([
    Row(name='Roni',age=27,date=168)]).toDF()
titi.show()
titi.filter((col('age')<29) & (col('date')>166)).show()

+---+----+----+
|age|date|name|
+---+----+----+
| 27| 168|Roni|
+---+----+----+



In [52]:
#Test Cell
toto = get_USReal("*","PM2_5")
toto.show()
a = 5
c = toto.select((col("_3")*5) + (col("_2"))).collect()[0][0]
print(c)

get_USPoint("*","PM2_5").show()

+---+-------------------+-------------------+-----+------------------+------------------+
| _1|                 _2|                 _3|   _4|                _5|                _6|
+---+-------------------+-------------------+-----+------------------+------------------+
|0.0|               -0.0|                2.0|false| 2041.969001666668|2041.9690006666679|
|0.0|               -0.0|                1.0|false|2041.9672149999992|2041.9671606666677|
|0.0|                0.0|                1.0|false|        2041.96987|2041.9702190000007|
|0.0|               -0.0|                2.0|false|2041.9705983333324|2041.9705973333323|
|0.0| -92.48982612442346|  188863.5055910349|false|2041.9705983333324|2041.9381623333343|
|0.0|               -0.0|                2.0|false|2041.9696883333322|2041.9622856666656|
|0.0|  757.3844987731238|-1546554.1889082526|false|2041.9696883333322|2041.9789306666667|
|0.0| -177.5252973528344| 362501.96194624814|false|2041.9453866666654|2041.9622856666656|
|0.0| 119.

In [48]:
def atInstant_USPoint(USPoint, t):
    if USPoint in colonnes_names : 
        df_operation = get_USPoint("*",USPoint)
        
        df_operation = df_operation.filter((col('_1')<= t) & (col('_2')>= t))
        df_operation = df_operation.withColumn('p2p1', (df_operation[3]) - df_operation[2])
        df_operation = df_operation.withColumn('t2t1', (df_operation[1]) - df_operation[0])   
        #valuee = df_operation.select(df_operation['valeur']).collect()
        #print(valuee)
        value = df_operation.select(df_operation['t2t1']).collect()[0][0]
        if(value != 0): 
            a = df_operation.select((df_operation['p2p1'])/(df_operation['t2t1'])).collect()[0][0]
            b = df_operation.select(((df_operation[2] * df_operation[1]) - (df_operation[3] * df_operation[0])) * df_operation['t2t1']).collect()[0][0]
            pos = ((a*t) + b)
            d = [(pos, t)]
            result = spark.createDataFrame(d, StructType([StructField('Cle_Pos', FloatType()), StructField('t', IntegerType())]))
        else :
            d = [(0.0, t)]
            result = spark.createDataFrame(d, StructType([StructField('Cle_Pos', FloatType()), StructField('t', IntegerType())]))
        return result.show()
atInstant_USPoint("PM2_5", 1573811410)
#a.show()

+------------+----------+
|     Cle_Pos|         t|
+------------+----------+
|1.15543168E8|1573811410|
+------------+----------+

